# Example File:
In this package, we show three examples: 
<ol>
    <li>4 site XY model</li>
    <li>4 site Transverse Field XY model with random coefficients</li>
    <li><b> Custom Hamiltonian from OpenFermion </b> </li>
</ol>

## Clone and Install The Repo via command line:
```
git clone https://github.com/kemperlab/cartan-quantum-synthesizer.git <br />
  
cd ./cartan-quantum-synthesizer/ <br />
  
pip install . <br />
````


# Building Custom Hamiltonians
In this example, we will use OpenFermion to generate a Hubbard Model Hamiltonian, then use the Jordan-Wigner methods of OpenFermion and some custom functions to feed the output into the Cartan-Quantum-Synthesizer package

## Step 1: Build the Hamiltonian in OpenFermion

In [33]:
from CQS.methods import *
from CQS.util.IO import tuplesToMatrix
import openfermion
from openfermion import FermionOperator

t = 1
U = 8
mu = 1
systemSize = 4 #number of qubits neeed
#2 site, 1D lattice, indexed as |↑_0↑_1↓_2↓_3>
#Hopping terms
H += -t*(FermionOperator('0^ 1') + FermionOperator('1^ 0') + FermionOperator('2^ 3') + FermionOperator('3^ 2'))
#Coulomb Terms
H += H + U*(FermionOperator('0^ 0 2^ 2') + FermionOperator('1^ 1 3^ 3'))
#Chemical Potential
H + -1* mu*(FermionOperator('0^ 0') + FermionOperator('1^ 1') + FermionOperator('2^ 2') + FermionOperator('3^ 3'))
print(H)

-1.0 [0^ 0] +
8.0 [0^ 0 2^ 2] +
-1.0 [0^ 1] +
-1.0 [1^ 0] +
-1.0 [1^ 1] +
8.0 [1^ 1 3^ 3] +
-1.0 [2^ 2] +
-1.0 [2^ 3] +
-1.0 [3^ 2] +
-1.0 [3^ 3]


In [34]:
#Jordan Wigner Transform
HPauli = openfermion.jordan_wigner(H)
print(HPauli)


(2+0j) [] +
(-0.5+0j) [X0 X1] +
(-0.5+0j) [Y0 Y1] +
(-1.5+0j) [Z0] +
(2+0j) [Z0 Z2] +
(-1.5+0j) [Z1] +
(2+0j) [Z1 Z3] +
(-0.5+0j) [X2 X3] +
(-0.5+0j) [Y2 Y3] +
(-1.5+0j) [Z2] +
(-1.5+0j) [Z3]


In [35]:
#Custom Function to convert OpenFermion operators to a format readable by CQS:
#Feel free to use or modify this code, but it is not built into the CQS package
def OpenFermionToCQS(H, systemSize):
        """
        Converts the Operators to a list of (PauliStrings)

        Args:
            H(obj): The OpenFermion Operator
            systemSize (int): The number of qubits in the system
        """
        stringToTuple  = {   
                        'X': 1,
                        'Y': 2,
                        'Z': 3
                    }
        opList = []
        coList = [] 
        for op in H.terms.keys(): #Pulls the operator out of the QubitOperator format
            coList.append(H.terms[op])
            opIndexList = []
            opTypeDict = {}
            tempTuple = ()
            for (opIndex, opType) in op:
                opIndexList.append(opIndex)
                opTypeDict[opIndex] = opType
            for index in range(systemSize):
                if index in opIndexList:
                    tempTuple += (stringToTuple[opTypeDict[index]],)
                else:
                    tempTuple += (0,)
            opList.append(tempTuple)
        return (coList, opList)
    
#The new format looks like:
print(OpenFermionToCQS(HPauli, systemSize))

([(-0.5+0j), (-0.5+0j), (-0.5+0j), (-0.5+0j), (2+0j), (-1.5+0j), (-1.5+0j), (2+0j), (-1.5+0j), (-1.5+0j), (2+0j)], [(2, 2, 0, 0), (1, 1, 0, 0), (0, 0, 2, 2), (0, 0, 1, 1), (0, 0, 0, 0), (0, 0, 3, 0), (3, 0, 0, 0), (3, 0, 3, 0), (0, 0, 0, 3), (0, 3, 0, 0), (0, 3, 0, 3)])


In [36]:
#Now, we can put all this together:

#Step 1: Create an Empty Hamiltonian Object
HubbardH = Hamiltonian(systemSize)

#Use Hamiltonian.addTerms to build the Hubbard model Hamiltonian:
HubbardH.addTerms(OpenFermionToCQS(HPauli, systemSize))
#This gives:
HubbardH.getHamiltonian(type='printText')
#There's an IIII term we would rather not deal with, so we can remove it like this:
HubbardH.removeTerm((0,0,0,0))
#This gives:
print('Idenity/Global Phase removed:')
HubbardH.getHamiltonian(type='printText')

#Be careful choosing an involution, because it might now decompose such that the Hamiltonian is in M:
try:
    HubbardC = Cartan(HubbardH)
except Exception as e:
    print('Default Even/Odd Involution does not work:')
    print(e)
print('countY does work through. g = ')
HubbardC = Cartan(HubbardH, involution='countY')
print(HubbardC.g)



(-0.5+0j) * YYII
(-0.5+0j) * XXII
(-0.5+0j) * IIYY
(-0.5+0j) * IIXX
(2+0j) * IIII
(-1.5+0j) * IIZI
(-1.5+0j) * ZIII
(2+0j) * ZIZI
(-1.5+0j) * IIIZ
(-1.5+0j) * IZII
(2+0j) * IZIZ
Idenity/Global Phase removed:
(-0.5+0j) * YYII
(-0.5+0j) * XXII
(-0.5+0j) * IIYY
(-0.5+0j) * IIXX
(-1.5+0j) * IIZI
(-1.5+0j) * ZIII
(2+0j) * ZIZI
(-1.5+0j) * IIIZ
(-1.5+0j) * IZII
(2+0j) * IZIZ
Default Even/Odd Involution does not work:
Invalid Involution. Please Choose an involution such that H ⊂ m
countY does work through. g = 
[(1, 2, 0, 0), (1, 2, 3, 0), (2, 1, 0, 0), (2, 1, 0, 3), (2, 1, 3, 0), (1, 2, 0, 3), (0, 0, 1, 2), (3, 0, 1, 2), (0, 0, 2, 1), (0, 3, 2, 1), (3, 0, 2, 1), (0, 3, 1, 2), (2, 2, 1, 2), (1, 1, 2, 1), (2, 2, 2, 1), (1, 1, 1, 2), (1, 2, 2, 2), (1, 2, 1, 1), (2, 1, 1, 1), (2, 1, 2, 2), (3, 3, 2, 1), (3, 3, 1, 2), (2, 1, 3, 3), (1, 2, 3, 3), (2, 2, 0, 0), (1, 1, 0, 0), (0, 0, 2, 2), (0, 0, 1, 1), (1, 1, 3, 3), (2, 2, 3, 3), (3, 3, 1, 1), (3, 3, 2, 2), (2, 2, 2, 2), (1, 1, 1, 1), (2, 2, 1, 1),